In [1]:
import pandas as pd
import numpy as np
import time

from requests import get
from bs4 import BeautifulSoup

from utils import lower_and_remove_diacritics, similar, check_row_similariy




Already scraped

artist_name = "Severina"
url = 'https://cuspajz.com/tekstovi-pjesama/izvodjac/severina.html'


artist_name = "Mia Dimšić" ##doesn't exist


artist_name = "Jelena Rozga"
url = 'https://cuspajz.com/tekstovi-pjesama/izvodjac/jelena-rozga.html'


artist_name = "Petar Grašo"
url = 'https://cuspajz.com/tekstovi-pjesama/izvodjac/petar-graso.html'


artist_name = "Đani Maršan"
url = 'https://cuspajz.com/tekstovi-pjesama/izvodjac/djani-marsan.html'


artist_name = "Aerodrom"
url = 'https://cuspajz.com/tekstovi-pjesama/izvodjac/aerodrom.html'


artist_name = "Alen Vitasović"
url = 'https://cuspajz.com/tekstovi-pjesama/izvodjac/alen-vitasovic.html'

In [2]:
artist_name = "Alen Vitasović"
url = 'https://cuspajz.com/tekstovi-pjesama/izvodjac/alen-vitasovic.html'

In [3]:
df = pd.read_csv(f"data/{artist_name}_tekstovinet_lyricstranslate_diskografija.csv")


In [4]:
df["Lyrics_lyricstranslate"].isnull().sum()

20

In [5]:
df[df["Lyrics_lyricstranslate"].isnull()]

,Song_ID,Artist,Artist_diskografija,Song_tekstovinet,Views_tekstovinet,Lyrics_tekstovinet,Url_tekstovinet,Writer_lyricstranslate,Song_lyricstranslate,EngTitle_lyricstranslate,...,Url_diskografija,Song_diskografija,Album_1_diskografija,Album_2_diskografija,Album_3_diskografija,Skladba_diskografija,Stihovi_diskografija,Aranžman_diskografija,Album_4_diskografija,Album_5_diskografija
2,H/8bQC7v_1_2,Alen Vitasović,Alen Vitasović,Ca mi sad ne smi rec (feat. Zdenko Hrsak),1705,"Vani je cvika, po krovu ledi tuce \r\nnema kuc...","https://tekstovi.net/2,2448,52349.html",NaN,NaN,NaN,...,https://diskografija.com/pjesma/ca-mi-sad-ne-s...,Ča mi sad ne smi reć,NaN,NaN,NaN,['Zdenko Hršak'],['Zdenko Hršak'],['Hrvoje Grčević'],NaN,NaN
3,H/8bQC7v_1_3,Alen Vitasović,Alen Vitasović,Cesta priko mora,5463,Ja san s tobon biti stija\r\n i po svitu san h...,"https://tekstovi.net/2,2448,47629.html",NaN,NaN,NaN,...,https://diskografija.com/pjesma/cesta-priko-mo...,Cesta priko mora,1997 - Come va?,2010 - Zlatna kolekcija,NaN,NaN,NaN,NaN,NaN,NaN
4,H/8bQC7v_1_4,Alen Vitasović,Alen Vitasović,Cvit,1052,Moglo je pasati cuda bolje\r\nvrimena je bilo ...,"https://tekstovi.net/2,2448,58376.html",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,H/8bQC7v_1_5,Alen Vitasović,Alen Vitasović,Dajla,403,Ovdje jutra mirisu na tebe \r\nkao cvijece kad...,"https://tekstovi.net/2,2448,61492.html",NaN,NaN,NaN,...,https://diskografija.com/pjesma/dajla-21718.htm,Dajla,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,H/8bQC7v_1_6,Alen Vitasović,Alen Vitasović,Groznjana grada (duet Scike),1726,Lipa mlada\r\nspod Groznjana grada\r\nreci nan...,"https://tekstovi.net/2,2448,49981.html",NaN,NaN,NaN,...,https://diskografija.com/pjesma/groznjana-grad...,Grožnjana grada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,H/8bQC7v_1_8,Alen Vitasović,Alen Vitasović,Ja ne gren,5051,Su mi rekli da je kasno i da moraju zaprit\r\n...,"https://tekstovi.net/2,2448,40441.html",NaN,NaN,NaN,...,https://diskografija.com/pjesma/ja-ne-gren-142...,Ja ne gren,2000 - Dora 2000,2000 - Ja ne gren,2010 - Zlatna kolekcija,['Robert Pilepić'],['Robert Pilepić'],NaN,NaN,NaN
9,H/8bQC7v_1_9,Alen Vitasović,Alen Vitasović,Ja san Istrijan,5575,Ja volin poj u Split\r\npar dani doli cu bit\r...,"https://tekstovi.net/2,2448,39390.html",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,H/8bQC7v_1_11,Alen Vitasović,Alen Vitasović,Kad ce vrime od kruha i grozja,1877,"Nis drugo ne triba \r\ni nis ne ceka, ajmo vec...","https://tekstovi.net/2,2448,47508.html",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,H/8bQC7v_1_12,Alen Vitasović,Alen Vitasović,Ko mi reces da,2923,Ca bin za te ucinija\r\nca bin za te da\r\npos...,"https://tekstovi.net/2,2448,40442.html",NaN,NaN,NaN,...,https://diskografija.com/pjesma/ko-mi-reces-da...,Ko mi rečeš da,1995 - Svi festivali,2010 - Zlatna kolekcija,NaN,NaN,NaN,NaN,NaN,NaN
13,H/8bQC7v_1_13,Alen Vitasović,Alen Vitasović,Lipe oci,2052,Ona mi je bila\r\nsve ca san prova\r\nma kamo ...,"https://tekstovi.net/2,2448,49982.html",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df.head()

,Song_ID,Artist,Artist_diskografija,Song_tekstovinet,Views_tekstovinet,Lyrics_tekstovinet,Url_tekstovinet,Writer_lyricstranslate,Song_lyricstranslate,EngTitle_lyricstranslate,...,Url_diskografija,Song_diskografija,Album_1_diskografija,Album_2_diskografija,Album_3_diskografija,Skladba_diskografija,Stihovi_diskografija,Aranžman_diskografija,Album_4_diskografija,Album_5_diskografija
0,H/8bQC7v_1_0,Alen Vitasović,Alen Vitasović,Ako tebi lazen ja,2417,"Mislim na tebe\r\nka iz spine curi vrime, brzo...","https://tekstovi.net/2,2448,42582.html",NaN,Ako tebi lažen ja,NaN,...,https://diskografija.com/pjesma/ako-tebi-lazen...,Ako tebi lažen ja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,H/8bQC7v_1_1,Alen Vitasović,Alen Vitasović,Bura,7517,"Bura, svo je grane slomila\r\ni tvoje je tilo ...","https://tekstovi.net/2,2448,32922.html",NaN,Bura,NaN,...,https://diskografija.com/pjesma/bura-2549.htm,Bura,1995 - Svi festivali,2008 - 10 Godina Melodija Hrvatskog Jadrana,2010 - Zlatna kolekcija,NaN,NaN,NaN,NaN,NaN
2,H/8bQC7v_1_2,Alen Vitasović,Alen Vitasović,Ca mi sad ne smi rec (feat. Zdenko Hrsak),1705,"Vani je cvika, po krovu ledi tuce \r\nnema kuc...","https://tekstovi.net/2,2448,52349.html",NaN,NaN,NaN,...,https://diskografija.com/pjesma/ca-mi-sad-ne-s...,Ča mi sad ne smi reć,NaN,NaN,NaN,['Zdenko Hršak'],['Zdenko Hršak'],['Hrvoje Grčević'],NaN,NaN
3,H/8bQC7v_1_3,Alen Vitasović,Alen Vitasović,Cesta priko mora,5463,Ja san s tobon biti stija\r\n i po svitu san h...,"https://tekstovi.net/2,2448,47629.html",NaN,NaN,NaN,...,https://diskografija.com/pjesma/cesta-priko-mo...,Cesta priko mora,1997 - Come va?,2010 - Zlatna kolekcija,NaN,NaN,NaN,NaN,NaN,NaN
4,H/8bQC7v_1_4,Alen Vitasović,Alen Vitasović,Cvit,1052,Moglo je pasati cuda bolje\r\nvrimena je bilo ...,"https://tekstovi.net/2,2448,58376.html",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Scraping cuspajz.com for song lyrics by artist

response = get(url)
html_soup = BeautifulSoup(response.text, 'html.parser')

# Get the list of songs
songs = html_soup.find_all('ul', class_='songList')



In [8]:
song_names = []
song_urls = []
for col_index, column in enumerate(songs):
    songs = column.find_all('li')
    for index, song in enumerate(songs):
        song_names.append(song.a.text)
        song_urls.append(song.a['href'])

In [9]:
similarity_threshold = 0.9
for row in df.iterrows():
    song_title = lower_and_remove_diacritics(row[1].Song_tekstovinet).split('(')[0]
    similarities = [similar(song_title, lower_and_remove_diacritics(s.split('(')[0])) for s in song_names]
    max_similarity = max(similarities)
    
    if max_similarity>similarity_threshold:
        cuspajz_name = song_names[similarities.index(max_similarity)] 
        song_url = song_urls[similarities.index(max_similarity)]
        url = 'https://cuspajz.com/'+song_url

        response = get(url)
        html_soup = BeautifulSoup(response.text, 'html.parser')

        song_lyrics = html_soup.find_all('p', class_='clearfix')
        lyrics =  song_lyrics[0].get_text()
        
        df.loc[row[0], "Lyrics_cuspajz"] = lyrics
        df.loc[row[0], "Song_cuspajz"] = cuspajz_name
        df.loc[row[0], "Url_cuspajz"] = url
        

        print("Lyrics successfully written to file for : " + song_title)
                      
        time.sleep(3)
        

Lyrics successfully written to file for : ako tebi lazen ja
Lyrics successfully written to file for : bura
Lyrics successfully written to file for : ja ne gren
Lyrics successfully written to file for : ja san istrijan
Lyrics successfully written to file for : jenu noc
Lyrics successfully written to file for : ko mi reces da
Lyrics successfully written to file for : od subote


In [10]:
wrong_rows = check_row_similariy(df, "Lyrics_tekstovinet", "Lyrics_cuspajz", threshold=0.7)

In [11]:
##set parameters as nan for wrong rows
for row in wrong_rows:
    df.loc[row[0], "Lyrics_cuspajz"] = np.nan
    df.loc[row[0], "Song_cuspajz"] = np.nan
    df.loc[row[0], "Url_cuspajz"] = np.nan

In [12]:
len(df)-len(df[["Lyrics_lyricstranslate", "Lyrics_cuspajz"]].dropna(how="all"))

16

In [13]:
df[df["Lyrics_lyricstranslate"].isnull() & df["Lyrics_cuspajz"].isnull()]

,Song_ID,Artist,Artist_diskografija,Song_tekstovinet,Views_tekstovinet,Lyrics_tekstovinet,Url_tekstovinet,Writer_lyricstranslate,Song_lyricstranslate,EngTitle_lyricstranslate,...,Album_2_diskografija,Album_3_diskografija,Skladba_diskografija,Stihovi_diskografija,Aranžman_diskografija,Album_4_diskografija,Album_5_diskografija,Lyrics_cuspajz,Song_cuspajz,Url_cuspajz
2,H/8bQC7v_1_2,Alen Vitasović,Alen Vitasović,Ca mi sad ne smi rec (feat. Zdenko Hrsak),1705,"Vani je cvika, po krovu ledi tuce \r\nnema kuc...","https://tekstovi.net/2,2448,52349.html",NaN,NaN,NaN,...,NaN,NaN,['Zdenko Hršak'],['Zdenko Hršak'],['Hrvoje Grčević'],NaN,NaN,NaN,NaN,NaN
3,H/8bQC7v_1_3,Alen Vitasović,Alen Vitasović,Cesta priko mora,5463,Ja san s tobon biti stija\r\n i po svitu san h...,"https://tekstovi.net/2,2448,47629.html",NaN,NaN,NaN,...,2010 - Zlatna kolekcija,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,H/8bQC7v_1_4,Alen Vitasović,Alen Vitasović,Cvit,1052,Moglo je pasati cuda bolje\r\nvrimena je bilo ...,"https://tekstovi.net/2,2448,58376.html",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,H/8bQC7v_1_5,Alen Vitasović,Alen Vitasović,Dajla,403,Ovdje jutra mirisu na tebe \r\nkao cvijece kad...,"https://tekstovi.net/2,2448,61492.html",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,H/8bQC7v_1_6,Alen Vitasović,Alen Vitasović,Groznjana grada (duet Scike),1726,Lipa mlada\r\nspod Groznjana grada\r\nreci nan...,"https://tekstovi.net/2,2448,49981.html",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,H/8bQC7v_1_11,Alen Vitasović,Alen Vitasović,Kad ce vrime od kruha i grozja,1877,"Nis drugo ne triba \r\ni nis ne ceka, ajmo vec...","https://tekstovi.net/2,2448,47508.html",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,H/8bQC7v_1_13,Alen Vitasović,Alen Vitasović,Lipe oci,2052,Ona mi je bila\r\nsve ca san prova\r\nma kamo ...,"https://tekstovi.net/2,2448,49982.html",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,H/8bQC7v_1_14,Alen Vitasović,Alen Vitasović,Lipo ime Hajdukovo,178,Od stare Plinare do naseg vrimena\r\nisli smo ...,"https://tekstovi.net/2,2448,64466.html",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,H/8bQC7v_1_15,Alen Vitasović,Alen Vitasović,Moja mila,3407,"Lipa moja, kadi si bila nocas\r\nceka san sam,...","https://tekstovi.net/2,2448,47613.html",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,H/8bQC7v_1_19,Alen Vitasović,Alen Vitasović,Nisi vise mala,18617,Poli mene pasiva \r\nkako da me ne pozna\r\nsv...,"https://tekstovi.net/2,2448,39391.html",NaN,NaN,NaN,...,2000 - Ja ne gren,2010 - Zlatna kolekcija,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df.to_csv(f'data/{artist_name}_tekstovinet_lyricstranslate_diskografija_cuspajz.csv', index=False)